<a href="https://colab.research.google.com/github/KashishOswal9/Automatic-License-plate-detection/blob/main/Automatic_License_plate_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install easyocr

In [4]:
import cv2
import numpy as np
import easyocr
import imutils
import os
from google.colab.patches import cv2_imshow


In [5]:
# Reorder points for perspective transform
def order_points(pts):
    pts = pts.reshape(4, 2)
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]  # top-left
    rect[2] = pts[np.argmax(s)]  # bottom-right
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # top-right
    rect[3] = pts[np.argmax(diff)]  # bottom-left
    return rect

def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    widthA = np.linalg.norm(br - bl)
    widthB = np.linalg.norm(tr - tl)
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.linalg.norm(tr - br)
    heightB = np.linalg.norm(tl - bl)
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([[0, 0], [maxWidth - 1, 0],
                    [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

def preprocess_plate(plate_img):
    gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
    kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
    sharp = cv2.filter2D(gray, -1, kernel)
    thresh = cv2.adaptiveThreshold(sharp, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 25, 15)
    return thresh



In [23]:
def detect_and_read_plates(frame, reader):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    morph = cv2.morphologyEx(bfilter, cv2.MORPH_CLOSE, kernel)
    edged = cv2.Canny(morph, 30, 200)

    contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:100]

    plates = []
    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.018 * peri, True)
        if len(approx) == 4:
            (x, y, w, h) = cv2.boundingRect(approx)
            aspect_ratio = w / float(h)
            area = cv2.contourArea(approx)
            if 2 < aspect_ratio < 6 and area > 1200:
                warped = four_point_transform(frame, approx)
                preprocessed = preprocess_plate(warped)
                results = reader.readtext(preprocessed)
                if results:
                    best_text = max(results, key=lambda r: r[2])
                    text = best_text[1]
                    confidence = best_text[2]
                    plates.append((text, confidence, approx))
    return plates

def draw_plates(frame, plates):
    for (text, conf, approx) in plates:
        pts = approx.reshape(4, 2)

        # Draw rectangle on plate
        for i in range(4):
            pt1 = tuple(pts[i])
            pt2 = tuple(pts[(i+1) % 4])
            cv2.line(frame, pt1, pt2, (0, 255, 0), 2)

        # Calculate center-bottom location for text
        min_y = np.max(pts[:,1]) + 40  # below plate
        min_x = int(np.mean(pts[:,0])) - 100  # center roughly
        label = f"{text}"

        # Draw large readable text below the car
        cv2.putText(frame, label, (min_x, min_y), cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 255, 0), 2, cv2.LINE_AA)

    return frame




def process_single_image(image_path, reader):
    img = cv2.imread(image_path)
    plates = detect_and_read_plates(img, reader)

    # Filter to keep only the most confident plate (if multiple)
    if plates:
        # Sort by confidence, keep only the best one
        plates = [max(plates, key=lambda x: x[1])]

    result = draw_plates(img, plates)
    cv2_imshow(result)  # Show in Colab
    cv2.imwrite("output_image.jpg", result)


def process_video(video_path, reader):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Cannot open video file: {video_path}")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    out = cv2.VideoWriter('output_video.mp4', fourcc, 20.0, (frame_width, frame_height))

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        plates = detect_and_read_plates(frame, reader)
        output_frame = draw_plates(frame.copy(), plates)
        out.write(output_frame)
        frame_count += 1
        if frame_count % 30 == 0:
            print(f"Processed {frame_count} frames...")

    cap.release()
    out.release()
    print("✅ Video processing done. Saved as output_video.mp4")


In [ ]:
def main():
    reader = easyocr.Reader(['en'])

    # Step 1️⃣ Image Output
    process_single_image('image1.jpg', reader)

    # Step 2️⃣ Video Output
    process_video('car1.mp4', reader)

main()



In [25]:
from google.colab import files
files.download("output_video.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>